# Smartfin Website Scrape
Smartfin ride data can be accessed through the Smartfin website through ride ids. Each Smartfin session has a unique ride id that we use to access the data of that session. The data to be accessed are the motion data and ocean data

Motion Data is data collected by the Smartfin's IMU sensors and take measurements of acceleration, rotation, and orientaion, each in the x, y, and z axis respectively.

Ocean Data records things like temperature, salinity, and pH, although the current generation of Smartfin only records temperature

The code in this notebook is used to add new smartfin session data to the Ride API database, so you won't actually have to web scrape any data manually for this project unless you just want to play around with the code. 

In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt

plt.rc("font", size=14) 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#from mpl_toolkits.basemap import Basemap


import os
import datetime
import pytz
import re

import peakutils
import statsmodels.api as sm

import requests


ride_ids = ['15692']


#%% Fin ID scraper
# Input fin ID, get all ride IDs
# base URL to which we'll append given fin IDs
# fin_url_base = 'http://surf.smartfin.org/fin/'

# Look for the following text in the HTML contents in fcn below
# str_id_ride = 'rideId = \'' # backslash allows us to look for single quote
# str_id_date = 'var date = \'' # backslash allows us to look for single quote

#%% Ride ID scraper


# Input ride ID, get ocean and motion CSVs
# Base URL to which we'll append given ride IDs
ride_url_base = 'https://surf.smartfin.org/ride/'

# Look for the following text in the HTML contents in fcn below to get csv id 
str_id_csv = 'img id="temperatureChart" class="chart" src="' 


def get_csv_from_ride_id(rid):
    
# step 1    
    # Build URL for each individual ride
    ride_url = ride_url_base+str(rid)
    print("ride_url: " + ride_url)
    
# step 2
    # query smartfin website to retrieve the ride's webpage in HTML  
    html_contents = requests.get(ride_url).text
#     print("html contents: ", html_contents)
    
    # Find CSV file location id in html page by csv file tag
    loc_csv_id = html_contents.find(str_id_csv)
#     print("loc_csv_id: ", loc_csv_id)
    
# step 3
    # log into smartfin website to get request authentication
    # Different based on whether user logged in with FB or Google
    offset_googleOAuth = [46, 114]
    offset_facebkOAuth = [46, 112]
    if html_contents[loc_csv_id+59] == 'f': # Facebook login
        off0 = offset_facebkOAuth[0]
        off1 = offset_facebkOAuth[1]
    else: # Google login
        off0 = offset_googleOAuth[0]
        off1 = offset_googleOAuth[1]

# step 4
    # use csv id and authentication offsets to build query string
    csv_id_longstr = html_contents[loc_csv_id+off0:loc_csv_id+off1]
#     print("csv_id_longstr: ", csv_id_longstr)
    
    # Stitch together full URL for CSV
    if ("media" in csv_id_longstr) & ("Calibration" not in html_contents): # other junk URLs can exist and break everything

# step 5
        # full urls to get csv file       
        ocean_csv_url = f'https://surf.smartfin.org/{csv_id_longstr}Ocean.CSV'
        motion_csv_url = f'https://surf.smartfin.org/{csv_id_longstr}Motion.CSV'
        
        print("ocean_csv_url: ", ocean_csv_url)
        print("motion_csv_url: ", motion_csv_url)
        print('\n\n')

# step 6
        # Go to ocean_csv_url and grab contents (theoretically, a CSV)
        ocean_df_small = pd.read_csv(ocean_csv_url, parse_dates = [0])
        motion_df_small = pd.read_csv(motion_csv_url, parse_dates = [0])


# step 7
        # 7a. add elasped column to show how much time has elapsed since first reading
        elapsed_timedelta = (ocean_df_small['UTC']-ocean_df_small['UTC'][0])
        ocean_df_small['elapsed'] = elapsed_timedelta/np.timedelta64(1, 's')
        
        print("motion_df_small raw: ", motion_df_small)
        print('\n\n')
        print("ocean_df_small raw: ", ocean_df_small)
        print('\n\n')


        
        # 7b. make the index of each df the timestamp
        if len(ocean_df_small) > 1:
            ocean_df_small.set_index('UTC', drop = True, append = False, inplace = True)
            motion_df_small.set_index('UTC', drop = True, append = False, inplace = True)
            
            print("ocean_df_small length pre upsample: ", len(ocean_df_small))
            print("motion_df_small length pre upsample: ", len(motion_df_small))
            
            # 7c. resample data to 33ms intervals (30 Hz)
            #May need to change this sampling interval:
            sample_interval = '33ms'
                        
            ocean_df_small_resample = ocean_df_small.resample(sample_interval).mean()
            motion_df_small_resample = motion_df_small.resample(sample_interval).mean()
    
            
            print('ocean_df_resample length: ', len(ocean_df_small_resample))
            print('motion_df_resample length: ', len(motion_df_small_resample))
            print('\n\n')
            
            # returns all rows that have values in latitude column           
            # No need to save many extra rows with no fix
            # motion_df_small = motion_df_small[~np.isnan(motion_df_small.Latitude)]
            
            return ocean_df_small_resample, motion_df_small_resample

    # if dataframe is empty, just return empty dataframe    
    else:
        ocean_df_small_resample = pd.DataFrame() # empty DF just so something is returned
        motion_df_small_resample = pd.DataFrame() 
        return ocean_df_small_resample, motion_df_small_resample
    
    
    
    
    
# actual script

appended_ocean_list = [] # list of DataFrames from original CSVs
appended_motion_list = []
appended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame

## Nested loops (for each fin ID, find all ride IDs, then build a DataFrame from all ride CSVs)
## (Here, ride IDS are either ocean or motion dataframes)
count_good_fins = 0      # number of dataframes with non empty data
    
# Loop over ride_ids and find CSVs
for rid in ride_ids:
    print("rid: ", rid)
    try:
        # runs code from function defined above
        new_ocean_df, new_motion_df = get_csv_from_ride_id(rid) # get given ride's CSV from its ride ID using function above
        
        # for each non empty df, append to list of already created dataframes        
        if not new_ocean_df.empty: # Calibration rides, for example
            
            # Append only if DF isn't empty. There may be a better way to control empty DFs which are created above
            appended_multiIndex.append(str(rid)) # build list to be multiIndex of future DataFrame
            appended_ocean_list.append(new_ocean_df)
            appended_motion_list.append(new_motion_df)
            
            count_good_fins += 1
        
    except: 
        print("Ride threw an exception!")
        #print("Ride ", rid, "threw an exception!")    

#%% Build the "Master" DataFrame
# keys for each diferent dataframe in the big dataframes
df_keys = tuple(appended_multiIndex) # keys gotta be a tuple, a list which data in it cannot be changed

# concatinate all dataframes in each list into one big dataframe
ocean_df = pd.concat(appended_ocean_list, keys = df_keys, names=['ride_id'])
motion_df = pd.concat(appended_motion_list, keys = df_keys, names = ['ride_id'])


##Here, maybe just use info from the motion_df and don't worry about ocean_df data for now.
##If you do want ocean_df data, look at how Phil was getting it from "July 10th and 11th Calibration" jupyter notebook file.
#We can also check to see if the surfboard was recording "in-water-freq" or 
#"out-of-water-freq" based on how many NaN values we see. 



# 7d. clear na values from dataframes
#Drop the latitude and longitude values since most of them are Nan:
print('motion df length pre na drop: ', len(motion_df))
motion_df_dropped = motion_df.drop(columns=['Latitude', 'Longitude'])

#Drop the NAN values from the motion data:
motion_df_dropped = motion_df_dropped.dropna(axis=0, how='any')
print('motion_df_dropped length post na drop: ', len(motion_df_dropped))
print('\n\n')

# finished clean dataframes
print('motion_df_dropped: ', motion_df_dropped)
print('ocean_df: ', ocean_df)
motion_df.head(10)

rid:  15692
ride_url: https://surf.smartfin.org/ride/15692
ocean_csv_url:  https://surf.smartfin.org/media/201811/google_105349665704999793400_0006667E229D_181109191556_Ocean.CSV
motion_csv_url:  https://surf.smartfin.org/media/201811/google_105349665704999793400_0006667E229D_181109191556_Motion.CSV



motion_df_small raw:                                     UTC        Time  IMU A1  IMU A2  IMU A3  \
0     2018-11-09 19:16:03.806000+00:00  1414742884     NaN     NaN     NaN   
1     2018-11-09 19:16:03.809000+00:00  1414742887   493.0    48.0   110.0   
2     2018-11-09 19:16:04.061000+00:00  1414743138   513.0    89.0    62.0   
3     2018-11-09 19:16:04.312000+00:00  1414743387   494.0    92.0    80.0   
4     2018-11-09 19:16:04.565000+00:00  1414743639   421.0   205.0  -104.0   
...                                ...         ...     ...     ...     ...   
22552 2018-11-09 20:38:14.334000+00:00  1419643689   501.0   -11.0    99.0   
22553 2018-11-09 20:38:14.500000+00:00  1419643854

Time  IMU A1  IMU A2  \
ride_id UTC                                                              
15692   2018-11-09 19:16:03.789000+00:00  1.414743e+09   493.0    48.0   
        2018-11-09 19:16:03.822000+00:00           NaN     NaN     NaN   
        2018-11-09 19:16:03.855000+00:00           NaN     NaN     NaN   
        2018-11-09 19:16:03.888000+00:00           NaN     NaN     NaN   
        2018-11-09 19:16:03.921000+00:00           NaN     NaN     NaN   
        2018-11-09 19:16:03.954000+00:00           NaN     NaN     NaN   
        2018-11-09 19:16:03.987000+00:00           NaN     NaN     NaN   
        2018-11-09 19:16:04.020000+00:00           NaN     NaN     NaN   
        2018-11-09 19:16:04.053000+00:00  1.414743e+09   513.0    89.0   
        2018-11-09 19:16:04.086000+00:00           NaN     NaN     NaN   

                                          IMU A3  IMU G1  IMU G2  IMU G3  \
ride_id UTC                                                                
15692   2018-11-09 19:16:03.789000+00:00   110.0    75.0  -124.0   -86.0   
        2018-11-09 19:16:03.822000+00:00     NaN     NaN     NaN     NaN   
        2018-11-09 19:16:03.855000+00:00     NaN     NaN     NaN     NaN   
        2018-11-09 19:16:03.888000+00:00     NaN     NaN     NaN     NaN   
        2018-11-09 19:16:03.921000+00:00     NaN     NaN     NaN     NaN   
        2018-11-09 19:16:03.954000+00:00     NaN     NaN     NaN     NaN   
        2018-11-09 19:16:03.987000+00:00     NaN     NaN     NaN     NaN   
        2018-11-09 19:16:04.020000+00:00     NaN     NaN     NaN     NaN   
        2018-11-09 19:16:04.053000+00:00    62.0    34.0   -36.0   -92.0   
        2018-11-09 19:16:04.086000+00:00     NaN     NaN     NaN     NaN   

                                          IMU M1  IMU M2  IMU M3   Latitude  \
ride_id UTC                                                                   
15692   2018-11-09 19:16:03.789000+00:00  -309.0   209.0    39.0  3285871.0   
        2018-11-09 19:16:03.822000+00:00     NaN     NaN     NaN        NaN   
        2018-11-09 19:16:03.855000+00:00     NaN     NaN     NaN        NaN   
        2018-11-09 19:16:03.888000+00:00     NaN     NaN     NaN        NaN   
        2018-11-09 19:16:03.921000+00:00     NaN     NaN     NaN        NaN   
        2018-11-09 19:16:03.954000+00:00     NaN     NaN     NaN        NaN   
        2018-11-09 19:16:03.987000+00:00     NaN     NaN     NaN        NaN   
        2018-11-09 19:16:04.020000+00:00     NaN     NaN     NaN        NaN   
        2018-11-09 19:16:04.053000+00:00  -320.0   194.0    38.0        NaN   
        2018-11-09 19:16:04.086000+00:00     NaN     NaN     NaN        NaN   

                                           Longitude  
ride_id UTC                                           
15692   2018-11-09 19:16:03.789000+00:00 -11725690.0  
        2018-11-09 19:16:03.822000+00:00         NaN  
        2018-11-09 19:16:03.855000+00:00         NaN  
        2018-11-09 19:16:03.888000+00:00         NaN  
        2018-11-09 19:16:03.921000+00:00         NaN  
        2018-11-09 19:16:03.954000+00:00         NaN  
        2018-11-09 19:16:03.987000+00:00         NaN  
        2018-11-09 19:16:04.020000+00:00         NaN  
        2018-11-09 19:16:04.053000+00:00         NaN  
        2018-11-09 19:16:04.086000+00:00         NaN